# Norwegian Sentiment Analysis

### Imports

In [ ]:
!pip install --upgrade --force-reinstall pandas numpy tensorflow keras

In [3]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.model_selection import train_test_split
import pandas as pd

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

### Import our own dataset
Datasettet er har data fra to veldig ulike kilder. Den første kilden er [NoReC, Norwegian Review Corpus](https://github.com/ltgoslo/norec) som inneholder norske anmeldelser med dens tilhørende terningkast. Vi har preprosessert dette fra før slik at 1 og 2 ble satt til 1, 3 og 4 ble satt til 3 og 5 og 6 ble satt til 6 for å henholdsvis ha tre klasser: negativ, nøytral og positiv. Disse anmeldelsene er veldig lange og inneholder god grammatisk norsk.  
Den andre kilden er vårt eget datasett hvor vi har brukt Twitter's API for å hente ut norske tweets. Disse ble soft-annotert ved bruk av en ordliste av negative/positive ord samt emojiene i tweeten. Vi fjernte så alle tweets som ikke inneholdte norske ord. Til slutt stemmet vi tweetsene.

In [4]:
pd.set_option('max_colwidth', 300)
data = pd.read_csv('smixed.csv')
data.sample(4)

,sentiment,text
64822,3,66 ’ 4 2 til famagust ett en flott overgang
14329,3,jordfarget romreise for høy vokal er vel strengt tatt noe av et luksusproblem i by larm sammenheng det er ikke desto mindre vanskelig å ignorere idet iampsyencefictions åpningslåt city of spades runger utover youngstorget på musikkfestens første dag særlig når feedback spøkelset flagrer faretrue...
61901,6,heldigvis har vi som samfunn beveg oss litt fra 1976 det er helt greit at ting som var kosh da ikk
105217,6,warcraft iii frozen throne vi ventet veldig lenge på warcraft iii og da spillet endelig kom i fjor sommer var det et svært hyggelig gjensyn med denne klassiske strategiserien siden vi gjorde oss ferdig med spillet har vi ventet på en tilleggspakke med nytt innhold og nå et år etter er frozen thr...


### Split in train and test set
Vi deler datasettet opp i et treningssett og et testsett til forholdet 80/20. I tillegg fjerner vi alle tall og flere tegn som hadde uheldigvis blitt med.

In [9]:
X = data["text"].str.replace('\d+', '').replace('–', '').replace('"', '')
y = data["sentiment"].div(2)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

print("Train-set size: ", len(x_train))
print("Test-set size:  ", len(x_test))

Train-set size:  108196
Test-set size:   27050


### Tokenizer
En maskinlæringsmodell kan bare lese tall, ikke tekst. Så her gjør vi om alle ordene til et tall (token) som henviser til en ordbok hvor det tallet befinner seg. Hvert eksempel (linje i datasettet) blir omgjort til en vektor, eller sekvens, som inneholder hvert tall (token).

In [8]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X)
tokenizer.word_index

{'og': 1,
 'er': 2,
 'i': 3,
 'det': 4,
 'som': 5,
 'en': 6,
 'på': 7,
 'med': 8,
 'å': 9,
 'av': 10,
 'til': 11,
 'for': 12,
 'den': 13,
 'har': 14,
 'at': 15,
 'de': 16,
 'men': 17,
 'ikke': 18,
 'et': 19,
 'om': 20,
 'seg': 21,
 'du': 22,
 'fra': 23,
 'vi': 24,
 'han': 25,
 'så': 26,
 'kan': 27,
 'også': 28,
 'jeg': 29,
 'blir': 30,
 'dette': 31,
 'var': 32,
 'ut': 33,
 'mer': 34,
 'noe': 35,
 'når': 36,
 'man': 37,
 'enn': 38,
 'denne': 39,
 'her': 40,
 'eller': 41,
 'der': 42,
 'hun': 43,
 'litt': 44,
 'opp': 45,
 'skal': 46,
 'selv': 47,
 'etter': 48,
 'får': 49,
 'sin': 50,
 'vil': 51,
 'noen': 52,
 'over': 53,
 'to': 54,
 'ikk': 55,
 'være': 56,
 'da': 57,
 'mye': 58,
 'gjør': 59,
 'bare': 60,
 'inn': 61,
 'helt': 62,
 'andre': 63,
 'the': 64,
 'alle': 65,
 'filmen': 66,
 'godt': 67,
 'må': 68,
 'nå': 69,
 '–': 70,
 'nok': 71,
 'år': 72,
 'både': 73,
 'alt': 74,
 'mange': 75,
 'ha': 76,
 'mot': 77,
 'ved': 78,
 'kommer': 79,
 'hva': 80,
 'få': 81,
 'god': 82,
 'mellom': 83,
 'f

In [17]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)
val = 2
print("Text: ", x_train[val])
print("Tokens: ", np.array(x_train_tokens[val]))

Text:  fødesjapp er et sammarbeid mellom #rema og #narves som skal løs pr
Tokens:  [  26  904    9  200  120  201    1   14  630 3012   20  166   17  132
  111]


### Padding and truncating
Modellen må få like lange eksempler hver gang grunnet matriseoperasjoner og videre. Derfor må vi først finne hvor mange ord/tokens setningene består av, og deretter finne gjennomsnittsverdi og makslengde. 

In [19]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
print("Average token length: ", np.mean(num_tokens))
print("Max token length: ", np.max(num_tokens))

Average token length:  89.52412640669594
Max token length:  21650


Vi ser at gjennomsnittlig lengde er 90 ord og maksimal er 21650 ord (Her må det ha skjedd en feil?). Vi bestemmer oss for en lengde alle eksempler må være på og deretter kutter vi eller padder vi i hvert eksempel ut i fra forholdet til den spesfikke lengden.  
  
Padding: [1, 2, 3, 4] -> [1, 2, 3, 4, 0, 0]  
Truncating: [1, 2, 3, 4, 5, 6, 7, 8] -> [1, 2, 3, 4, 5, 6]  
Her har vi brukt pad='post' som legger til/fjerner på slutten.

In [22]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
print("Max tokens allowed: ", max_tokens)
print("Covers dataset amount: ", np.sum(num_tokens < max_tokens) / len(num_tokens))

Max tokens allowed:  660
Covers dataset amount:  0.9824911642488502


In [28]:
pad = 'pre'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)
print("Uten padding:\n", np.array(x_train_tokens[2]))
print("Med padding:\n", x_train_pad[2])

Uten padding:
 [  26  904    9  200  120  201    1   14  630 3012   20  166   17  132
  111]
Med padding:
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0 